In [39]:
import gensim
import pickle
from nltk.stem import WordNetLemmatizer
import json
import re
from nltk.tokenize import word_tokenize
from nltk.sentiment.util import mark_negation
import nltk
import os
import utils
from random import shuffle
import time

## Constants

In [2]:
eval_data_file='data/eval_data.txt'
eval_words_file='data/eval_data.txt.td'
lemmatizer=WordNetLemmatizer()


## Read Data

In [3]:
reader=open(eval_words_file,'r')
eval_words_list=[]
for line in reader:
    tok_line=line.strip().split(" ")
    x=len(tok_line)
    for i in range(x):
        tok_line[i]=tok_line[i].split(":")[-1]
    eval_words_list.append(tok_line)
print(len(eval_words_list))
reader.close()

21918


In [4]:
reader=open(eval_data_file,'r')
context_list=[]
y=[]

for line in reader:
    tok_line=line.strip().split("<<target>>")
    context=gensim.utils.simple_preprocess(tok_line[0],min_len=1)
    x=len(context)
    
    y.append(tok_line[1].split(":")[-1])
    context2=gensim.utils.simple_preprocess(tok_line[1])

    if len(context2)>0:
        context2.pop()
    
    context_list.append(context)
    context_list.append(context2)
print(len(context_list),len(y))
print(context_list[0],y[0])
print(context_list[1],y[0])

43836 21918
['the', 'particulars', 'of', 'your', 'plans', 'i', 'neither'] know
['or', 'seek', 'to', 'know'] know


In [191]:
eval_words_list=utils.lemmatize(eval_words_list)
context_list=utils.lemmatize(context_list)
y=utils.lemmatize_1d(y)

10000
20000
10000
20000
30000
40000


## Load Model

In [5]:
def write_file(arr,file):
    with open(file,'w') as f:
        for line in arr:
            st=" ".join([str(x) for x in line])
            print(st,file=f)
    f.close()

In [46]:
def func(func,save_path,flag=False,hits=2000,context_win=2,upto=2000):

    mrr=0
    mr=0
    ranking=[]

    if flag==True:
        upto=len(eval_words_list)

    for i in range(upto):
        if i%100==0:
            print(mrr/(i+1),mr*1.0/(i+1),i)
        context1=context_list[2*i]
        context2=context_list[2*i+1]
        x1=len(context1)
        x2=len(context2)
        if x1>context_win:
            context1=context1[x1-context_win:x1]
        if x2>context_win:
            context2=context2[0:context_win]
        
        context=context1+context2
        word=y[i]
        val=func(context,topn=hits)
        local_rank=[]
        mapper={}
        mapper_count={}
        for w in eval_words_list[i]:
            mapper[w]=0
            if w not in mapper_count:
                mapper_count[w]=0
            mapper_count[w]+=1

        count=1
        if val is not None:
            for j in range(len(val)):
                if val[j][0] in mapper:
                    mapper[val[j][0]]=count
                    count+=mapper_count[val[j][0]]

        temp=[]
        for w in eval_words_list[i]:
            if mapper[w]==0:
                temp.append(w)
        shuffle(temp)
        
        for w in temp:
            if mapper[w]==0:
                mapper[w]=count
                count+=mapper_count[w]

        for w in eval_words_list[i]:
            local_rank.append(mapper[w])
            mapper[w]+=1
        ranking.append(local_rank)

        if word not in mapper:
            print (word)
            print (eval_words_list[i])
            print (mapper)
        
        mr+=(mapper[word]-1)
        mrr+=1.0/(mapper[word]-1)
    write_file(ranking,save_path)
    return ranking

In [47]:
def run_eval(path,output,flag=False,hits=2000,context_win=1):
    model_path=path
    model=utils.load_pickle(model_path)
    x=time.time()
    
    t=func(model.predict_output_word,output,flag=flag,hits=hits,context_win=context_win)
    print(time.time()-x)
    return t

In [48]:
def getC(w1,w2,ds):
    key=w1+"_"+w2
    if key in ds:
        return ds[key]
    return 0
    
def counter(ds,word,context,begin):
    x=len(context)
    count=0.0
    wt=1.0

    if begin==1:
        for i in range(min(x,2)):
            count+=getC(word,context[i],ds)*wt
            wt/=2
    else:
        for i in range(x-1,max(0,x-3),-1):
            count+=getC(word,context[i],ds)*wt
            wt/=2
    return count

def rerank(rank_list,context_list,eval_words_list,rr=30):
    le=len(rank_list)
    aux_ds=utils.load_pickle(filename="aux_ds.pkl")
    for i in range(le):
        imp_words=[]
        lee=len(rank_list[i])
        for j in range(lee):
            if rank_list[i][j]<rr:
                imp_words.append((eval_words_list[i][j],rank_list[i][j],j))
        new_lis=[]
        for w in imp_words:
            word=w[0]
            rank=w[1]
            pos=w[2]
            count=0
            count+=counter(aux_ds,word,context_list[2*i],0)
            count+=counter(aux_ds,word,context_list[2*i+1],1)
            new_lis.append((count,-rank,pos))
#         print(new_lis)
        new_lis=sorted(new_lis,reverse=True)
#         print(new_lis)
        c=0
        for entry in new_lis:
            c+=1
            pos=entry[2]
            rank_list[i][pos]=c
    write_file(arr=rank_list,file="rerank.txt")
            

In [49]:
def pt(t):
    for i in range(5):
        print(context_list[2*i],context_list[2*i+1],y[i])
        temp=list(zip(t[i],eval_words_list[i]))
        print(sorted(temp)[0:20])
        print()

In [50]:
# model_path='./sg_emb150_ep10.pkl'
# out=run_eval(model_path,'out.txt',flag=False,hits=2000,context_win=1)
# pt(out)

In [51]:
# model_path='./sg_emb150_ep10.pkl'
# out=run_eval(model_path,'out.txt',flag=False,hits=2000,context_win=2)
# pt(out)

In [54]:
model_path='./sg_pret_emb300_ep10.pkl'
out=run_eval(model_path,'out.txt',flag=True,hits=2000,context_win=2)
print(len(out))
pt(out)

0.0 0.0 0
0.2893625043913157 28.178217821782177 100
0.2570734279060892 31.3681592039801 200
0.24575175102608496 33.42857142857143 300
0.23649064294920444 34.66334164588529 400
0.22798413002480464 35.76646706586826 500
0.23281548155980353 35.10316139767055 600
0.2321034834367869 34.35805991440799 700
0.2314730952981474 34.9250936329588 800
0.23645934318669148 33.983351831298556 900
0.23567005498937463 33.952047952047955 1000
0.23605105236297533 34.055404178019984 1100
0.24114337646201853 33.34221482098251 1200
0.24596899546720666 33.21137586471944 1300


/home/cse/btech/cs1150210/anaconda3/lib/python3.6/site-packages/gensim/models/word2vec.py:859: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")


0.24394060750067473 33.50535331905782 1400
0.2465289249063171 33.32644903397735 1500
0.24631599152540792 32.74640849469082 1600
0.248062029914419 32.37683715461493 1700
0.248349113693676 32.13659078289839 1800
0.2489570046751847 31.968437664387164 1900
0.25131121418166985 31.71864067966017 2000
0.25081759334523934 31.53069966682532 2100
0.2495029373357731 31.825533848250796 2200
0.25297854479351445 31.38200782268579 2300
0.2527052173850014 31.369012911286962 2400
0.25612493906092426 31.12794882047181 2500
0.25616470955922266 31.27258746635909 2600
0.25399851106022103 31.360236949278047 2700
0.25157947672290143 31.63977151017494 2800
0.2500784740248328 31.9617373319545 2900
0.24836564972169697 31.899366877707433 3000
0.247069635233168 32.247017091260886 3100
0.24742564702412356 32.223992502343016 3200
0.24766151666182457 31.97425022720388 3300
0.2498996948968053 31.81417230226404 3400
0.24968043600605705 31.72693516138246 3500
0.2489214366277218 31.8014440433213 3600
0.24941252023613253

0.2838286467553957 26.03561400916211 20300
0.28367432075610594 26.037792265085045 20400
0.2841419835844938 26.006048485439734 20500
0.28445901446875094 25.97534100286394 20600
0.2843427413420766 25.996328679773924 20700
0.2842645174808241 25.979856737656842 20800
0.2843098858484436 25.941773120903306 20900
0.28456195375683313 25.937907718680062 21000
0.28462727777904456 25.931330268707644 21100
0.28461593381610745 25.921748974104997 21200
0.2848083027379506 25.896155110088728 21300
0.2848879222704633 25.88790243446568 21400
0.2849832015086042 25.905864843495653 21500
0.2849999715514281 25.900930512476275 21600
0.2850722571372123 25.8918022210958 21700
0.2851735038974799 25.895417641392598 21800
0.28543201821856745 25.920231952878865 21900
352.5842888355255
21918
['the', 'particulars', 'of', 'your', 'plans', 'i', 'neither'] ['or', 'seek', 'to', 'know'] know
[(1, 'do'), (2, 'think'), (3, 'know'), (4, 'see'), (5, 'get'), (6, 'go'), (7, 'say'), (8, 'find'), (9, 'lose'), (10, 'knew'), (11, 

In [55]:
print(len(out))
rerank(out,context_list=context_list,eval_words_list=eval_words_list,rr=1000)

21918


In [199]:
model_path='./sg_pret_emb300_ep10.pkl'
out=run_eval(model_path,'out.txt',flag=False,hits=2000,context_win=2)
pt(out)

0.0 0.0 0
0.2893944782262512 25.237623762376238 100
0.2572403744650303 29.601990049751244 200
0.2459740348229016 32.21926910299003 300
0.23662566384313344 33.576059850374065 400
0.22811217196984215 34.510978043912175 500
0.233049795218528 33.15640599001664 600
0.2326048361356494 31.80599144079886 700
0.23189910073620068 32.45692883895131 800
0.23686077502706493 31.7857935627081 900
0.23600255860732838 31.929070929070928 1000
0.2363320654030125 32.473206176203455 1100
0.24141425782336312 31.825978351373855 1200
0.2463044484498575 31.448885472713297 1300


/home/cse/btech/cs1150210/anaconda3/lib/python3.6/site-packages/gensim/models/word2vec.py:859: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")


0.2442056183290529 31.705924339757317 1400
0.2468021207520262 31.614257161892073 1500
0.2465860848253774 31.195502810743285 1600
0.2483167239071588 30.951793062904173 1700
0.24858488105880908 30.856191004997225 1800
0.24914825817683459 31.048395581273013 1900
CPU times: user 6min 7s, sys: 1min 21s, total: 7min 29s
Wall time: 35.7 s
['the', 'particulars', 'of', 'your', 'plans', 'i', 'neither'] ['or', 'seek', 'to', 'know'] know
[(1, 'do'), (2, 'get'), (3, 'lose'), (4, 'go'), (5, 'see'), (6, 'come'), (7, 'think'), (8, 'know'), (9, 'find'), (10, 'appear'), (11, 'say'), (12, 'expect'), (13, 'take'), (14, 'give'), (15, 'leave'), (16, 'be'), (17, 'make'), (18, 'have'), (19, 'remove'), (20, 'knew')]
['while', 'i', 'am', 'very', 'anxious', 'that', 'any', 'great', 'disaster', 'or', 'the', 'capture', 'of', 'our', 'men', 'in', 'great', 'numbers'] ['be', 'avoided', 'know', 'these', 'points', 'are', 'less', 'likely', 'to', 'escape', 'your', 'attention', 'than', 'they', 'would', 'be', 'mine'] shall
[

### Lemm

In [185]:
model_path='./lemm_sg_emb200_ep10.pkl'
out=run_eval(model_path,'out.txt',flag=False,hits=2000,context_win=2)
pt(out)

0.0 0.0 0
0.26391612477753434 27.128712871287128 100
0.2376854486029815 30.199004975124378 200
0.2217697262875314 31.259136212624586 300
0.21616044145629432 31.25935162094763 400
0.2070474066972291 32.187624750499005 500
0.2120257312157065 31.111480865224625 600
0.20839003355376629 30.03994293865906 700
0.20771398907207983 31.18227215980025 800
0.21287780316937027 30.200887902330745 900
0.21344547699402405 30.285714285714285 1000
0.21301072857546782 30.52043596730245 1100
0.2181378429937186 29.98001665278934 1200
0.2230251622775511 29.63182167563413 1300


/home/cse/btech/cs1150210/anaconda3/lib/python3.6/site-packages/gensim/models/word2vec.py:859: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")


0.22219956340404148 30.11277658815132 1400
0.22650734193442826 30.109926715522985 1500
0.2259671032940872 29.75202998126171 1600
0.22744967892824336 29.59376837154615 1700
0.22904959248349605 29.56857301499167 1800
0.2278908114456888 29.56233561283535 1900
CPU times: user 5min 10s, sys: 1min 10s, total: 6min 20s
Wall time: 30.4 s
['the', 'particular', 'of', 'your', 'plan', 'i', 'neither'] ['or', 'seek', 'to', 'know'] know
[(1, 'do'), (2, 'get'), (3, 'lose'), (4, 'go'), (5, 'see'), (6, 'come'), (7, 'think'), (8, 'know'), (9, 'find'), (10, 'appear'), (11, 'say'), (12, 'expect'), (13, 'take'), (14, 'give'), (15, 'leave'), (16, 'be'), (17, 'make'), (18, 'have'), (19, 'remove'), (20, 'knew')]
['while', 'i', 'am', 'very', 'anxious', 'that', 'any', 'great', 'disaster', 'or', 'the', 'capture', 'of', 'our', 'men', 'in', 'great', 'number'] ['be', 'avoided', 'know', 'these', 'point', 'are', 'le', 'likely', 'to', 'escape', 'your', 'attention', 'than', 'they', 'would', 'be', 'mine'] shall
[(1, 'wou

In [192]:
model_path='./lemm_sg_emb300_ep10.pkl'
out=run_eval(model_path,'out.txt',flag=False,hits=2000,context_win=2)
pt(out)

0.0 0.0 0
0.25688693696160253 23.07920792079208 100
0.23925517310530106 28.29353233830846 200
0.22493485082417478 30.837209302325583 300
0.2162244229882309 32.71321695760599 400
0.21029776755662932 32.82235528942116 500
0.21343685875401025 31.948419301164726 600
0.21098447743034146 31.40085592011412 700
0.2086222644706398 32.07116104868914 800
0.21457922495151247 31.519422863485016 900
0.21478662947921515 31.3996003996004 1000
0.21514307230787305 31.819255222524976 1100
0.22012835862880784 30.896752706078267 1200
0.22382954537927022 30.911606456571867 1300


/home/cse/btech/cs1150210/anaconda3/lib/python3.6/site-packages/gensim/models/word2vec.py:859: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")


0.22331252935284046 31.37330478229836 1400
0.22626013597101433 31.5236508994004 1500
0.2254713290461498 31.245471580262336 1600
0.22660964028781114 30.95767195767196 1700
0.22705867404726635 30.89339255968906 1800
0.22597835221040954 30.711730668069436 1900
CPU times: user 5min 12s, sys: 1min 7s, total: 6min 19s
Wall time: 29.9 s
['the', 'particular', 'of', 'your', 'plan', 'i', 'neither'] ['or', 'seek', 'to', 'know'] know
[(1, 'do'), (2, 'get'), (3, 'lose'), (4, 'go'), (5, 'see'), (6, 'come'), (7, 'think'), (8, 'know'), (9, 'find'), (10, 'appear'), (11, 'say'), (12, 'expect'), (13, 'take'), (14, 'give'), (15, 'leave'), (16, 'be'), (17, 'make'), (18, 'have'), (19, 'remove'), (20, 'knew')]
['while', 'i', 'am', 'very', 'anxious', 'that', 'any', 'great', 'disaster', 'or', 'the', 'capture', 'of', 'our', 'men', 'in', 'great', 'number'] ['be', 'avoided', 'know', 'these', 'point', 'are', 'le', 'likely', 'to', 'escape', 'your', 'attention', 'than', 'they', 'would', 'be', 'mine'] shall
[(1, 'wou

In [193]:
model_path='./lemm_pre_sg_emb300_ep10.pkl'
out=run_eval(model_path,'out.txt',flag=False,hits=2000,context_win=2)
pt(out)

0.0 0.0 0
0.2676649835511454 24.752475247524753 100
0.24828999576880043 26.154228855721392 200
0.2303409385191886 28.205980066445182 300
0.22314654114821855 28.453865336658353 400
0.21036206838861563 28.201596806387226 500
0.213716530519465 27.183028286189685 600
0.21143448900197412 26.024251069900142 700
0.21092401311474912 26.436953807740323 800
0.21800731786213765 25.964483906770255 900
0.22038235373743267 25.829170829170828 1000
0.21948239261414645 26.26975476839237 1100
0.22326402501897627 26.074937552039966 1200
0.2276445056252417 25.983858570330515 1300


/home/cse/btech/cs1150210/anaconda3/lib/python3.6/site-packages/gensim/models/word2vec.py:859: UserWarning: All the input context words are out-of-vocabulary for the current model.
  warnings.warn("All the input context words are out-of-vocabulary for the current model.")


0.22830395610309423 26.422555317630263 1400
0.2307463904850185 26.421718854097268 1500
0.23027949509473006 26.14303560274828 1600
0.23200635779432546 25.890064667842445 1700
0.23226842778203236 25.932259855635756 1800
0.23162991790499343 26.042083114150447 1900
CPU times: user 5min 46s, sys: 1min 17s, total: 7min 3s
Wall time: 33.7 s
['the', 'particular', 'of', 'your', 'plan', 'i', 'neither'] ['or', 'seek', 'to', 'know'] know
[(1, 'do'), (2, 'get'), (3, 'lose'), (4, 'go'), (5, 'see'), (6, 'come'), (7, 'think'), (8, 'know'), (9, 'find'), (10, 'appear'), (11, 'say'), (12, 'expect'), (13, 'take'), (14, 'give'), (15, 'leave'), (16, 'be'), (17, 'make'), (18, 'have'), (19, 'remove'), (20, 'knew')]
['while', 'i', 'am', 'very', 'anxious', 'that', 'any', 'great', 'disaster', 'or', 'the', 'capture', 'of', 'our', 'men', 'in', 'great', 'number'] ['be', 'avoided', 'know', 'these', 'point', 'are', 'le', 'likely', 'to', 'escape', 'your', 'attention', 'than', 'they', 'would', 'be', 'mine'] shall
[(1, 

In [14]:
def aa(a):
    return 2*a

In [15]:
%time a=aa(5)
print(a)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 16.2 µs
10


In [213]:
def pt(t):
    for i in range(5):
        print(context_list[2*i],context_list[2*i+1],y[i])
        temp=list(zip(t[i],eval_words_list[i]))
        print(sorted(temp)[0:20])
        print()